# Imports

In [1]:
!pip install git+https://github.com/AlvaroJoseLopes/sequence2sequence-models
!pip install spacy -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AlvaroJoseLopes/sequence2sequence-models to /tmp/pip-req-build-z7y3xn17
  Running command git clone --filter=blob:none --quiet https://github.com/AlvaroJoseLopes/sequence2sequence-models /tmp/pip-req-build-z7y3xn17
  Resolved https://github.com/AlvaroJoseLopes/sequence2sequence-models to commit 63e0407e21f9d1d5fcacbc50ff300ad23e3b63cb
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.2 MB/s eta 0:00:00
  Created wheel for seq2seq-models: filename=seq2seq_models-0.0.1-py3-none-any.whl size=16299 sha256=3fd65f7192bbe34faa6d882dc1dc90a08e2681bb00f14f0a534ab107e30f0129
  Stored in directory: /tmp/pip-ephem-wheel-cache-t0aqmu3a/wheels/1c/7d/fa/bed29ef3b79ad38318eb71f5111a30d2cf83d66ba77101c01e
Successfully built seq2seq-models
  Attemp

In [2]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 89.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.2 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.1
    Uninstalling en-core-web-sm-3.4.1:
      Successfully uninstalled en-core-web-sm-3.4.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Training model

In [3]:
!mkdir models/

In [8]:
from seq2seq_models import seq2seq_LSTM as seq2seq
seq2seq.train_model_script(
    ('-src', 'en',
     '-trg', 'de',
     '-f', 'models/model_20_12300.pt',
     '-ep', '5',
     '-bs', '128',
     '-s', '12300')
)

Chosen device cuda


/usr/local/lib/python3.8/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(


Source vocab size: 10837
target vocab size: 19214
The model has 24,906,254 trainable parameters
Epoch: 01 
	Time: 1m 17s
	Train Loss: 5.539 | Train PPL: 254.305
	 Val. Loss: 5.561 |  Val. PPL: 259.991
Epoch: 02 
	Time: 1m 15s
	Train Loss: 4.846 | Train PPL: 127.264
	 Val. Loss: 5.449 |  Val. PPL: 232.516
Epoch: 03 
	Time: 1m 15s
	Train Loss: 4.566 | Train PPL:  96.158
	 Val. Loss: 5.335 |  Val. PPL: 207.519
Epoch: 04 
	Time: 1m 15s
	Train Loss: 4.285 | Train PPL:  72.624
	 Val. Loss: 5.161 |  Val. PPL: 174.424
Epoch: 05 
	Time: 1m 16s
	Train Loss: 4.092 | Train PPL:  59.848
	 Val. Loss: 5.088 |  Val. PPL: 162.074


# Avaliando embeddings das palavras

In [9]:
import torch
import pandas as pd
model = torch.load('models/model_20_12300.pt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Encoder

In [10]:
word_emb_en = seq2seq.get_word_embeddings(model, 'encoder', 'en', device)

In [11]:
# # temp
# lang_vocab = torch.load('vocabs/vocab_en.pth')
# matrix = model.get_embeddings_matrix('encoder').cpu().detach().numpy()
# word_embeddings = {}
# for token, embedding in enumerate(list(matrix)):
#     word_embeddings[lang_vocab.lookup_token(token)] = embedding
df_emb_en = pd.DataFrame.from_dict(word_emb_en, orient='index')

In [12]:
df_emb_en.shape

(10837, 256)

In [13]:
df_emb_en.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
<unk>,0.063939,-0.028902,0.056993,0.076601,-0.005598,-0.024722,0.068028,-0.009198,-0.030310,0.011572,...,0.048634,-0.067992,-0.042325,-0.008926,-0.054196,-0.060672,-0.038773,-0.076559,0.031633,0.001078
<pad>,0.025666,0.131593,0.033062,-0.061482,-0.144928,-0.122375,0.095551,0.094858,-0.084277,0.023836,...,0.067133,0.031875,-0.070262,-0.066198,-0.144842,0.045534,-0.096067,-0.128586,-0.074642,0.034505
<bos>,0.072650,-0.003381,0.077596,0.008755,0.015723,0.024204,-0.006869,-0.066496,0.018050,0.064313,...,-0.004506,0.040160,-0.031858,0.003418,0.042781,-0.104305,0.063362,0.078510,0.012066,0.005445
<eos>,0.121334,-0.025020,-0.057420,-0.002877,0.025437,0.094305,-0.050990,-0.042043,0.070418,0.119060,...,-0.011671,-0.060907,0.084404,0.043250,0.005974,-0.172317,0.034559,0.123608,0.057251,-0.031952
Two,0.071522,-0.322144,0.229077,-0.168940,0.322002,0.144016,-0.106117,-0.208939,0.094987,0.098780,...,-0.240993,-0.013775,-0.230093,0.209014,0.283308,0.043951,0.155411,0.267554,0.161529,0.070782


In [14]:
from sklearn.manifold import TSNE
tsne = TSNE(metric = 'cosine')
sample = df_emb_en.sample(2000)
reducedWordVecs = tsne.fit_transform(sample)

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


In [15]:
emb_reduced = {}
for idx, token in enumerate(sample.index):
  emb_reduced[token] = reducedWordVecs[idx]
df_emb_reduced = pd.DataFrame.from_dict(
        data = emb_reduced,
        orient = 'index',
        columns = ['x', 'y']
    )
df_emb_reduced['word'] = df_emb_reduced.index
df_emb_reduced.head()

,x,y,word
moutnain,13.557753,4.931347,moutnain
bulb,7.078380,-8.256011,bulb
labels,9.720503,9.145370,labels
turbulent,22.407784,-0.997724,turbulent
2012,-4.275962,9.095164,2012


In [16]:
# Configurando a ferramenta interativa com o notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

def interativePlot(df, title = None, xlabel = None, ylabel = None):
    # Instancia uma ColumnDAtaSource do Bokeh usando o dataframe
    data = ColumnDataSource(df)

    # Cria o plot e configura:
    # título, dimensões e ferramentas interativas
    plot = figure(
        title= title,
        plot_width=800,
        plot_height=800,
        tools=(
            'pan, wheel_zoom, box_zoom,'
            'box_select, reset',
            HoverTool(tooltips = '@word')
            ),
        active_scroll='wheel_zoom',
        x_axis_label = xlabel,
        y_axis_label = ylabel
        )

    # Desenha os circulos representado as palavras
    plot.circle(
        'x',
        'y',
        source=data,
        color='blue',
        line_alpha=0.2,
        fill_alpha=0.1,
        size=10,
        hover_line_color='black'
        )

    # Configuração de outros detalhes visuais
    plot.xaxis.visible = False
    plot.yaxis.visible = False
    plot.grid.grid_line_color = None
    plot.outline_line_color = None


    show(plot)

In [17]:
interativePlot(df_emb_reduced)

## Decoder

In [18]:
word_emb_de = seq2seq.get_word_embeddings(model, 'decoder', 'de', device)
df_emb_de = pd.DataFrame.from_dict(word_emb_de, orient='index')

In [19]:
from sklearn.manifold import TSNE
tsne = TSNE(metric = 'cosine')
sample = df_emb_de.sample(2000)
reducedWordVecs = tsne.fit_transform(sample)

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


In [20]:
emb_reduced = {}
for idx, token in enumerate(sample.index):
  emb_reduced[token] = reducedWordVecs[idx]
df_emb_reduced = pd.DataFrame.from_dict(
        data = emb_reduced,
        orient = 'index',
        columns = ['x', 'y']
    )
df_emb_reduced['word'] = df_emb_reduced.index
df_emb_reduced.head()

,x,y,word
Großmütter,-4.019757,-3.959334,Großmütter
Halbschatten,-6.617657,8.885860,Halbschatten
Falken,-1.248842,-9.158227,Falken
seines,2.735357,-9.616122,seines
abgelenkt,-6.522126,7.984297,abgelenkt


In [21]:
interativePlot(df_emb_reduced)

# Avaliando embedding das sentencas

## Exemplo 1

In [22]:
examples = ['Mary admires John',
 'Mary is in love with John',
 'Mary respects John',
 'John admires Mary',
 'John is in love with Mary',
 'John respects Mary']

In [23]:
h,c = seq2seq.get_sentences_embeddings(model, examples, 'en', device)

/usr/local/lib/python3.8/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [24]:
sentences_emb = ((h+c)/2).mean(0)
sentences_emb = sentences_emb.cpu().detach().numpy()
sentences_emb.shape

(6, 512)

In [25]:
from sklearn.manifold import TSNE
tsne = TSNE(metric = 'cosine')
reduced_sentences_emb = tsne.fit_transform(sentences_emb)
reduced_sentences_emb.shape

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


(6, 2)

In [26]:
sentences_emb = {}
for idx, sentence in enumerate(examples):
  sentences_emb[sentence] = reduced_sentences_emb[idx]
df_sentences_emb = pd.DataFrame.from_dict(data=sentences_emb, columns=['x', 'y'], orient='index')

In [27]:
df_sentences_emb['word'] = df_sentences_emb.index
df_sentences_emb.head()

,x,y,word
Mary admires John,-8.091329,62.217793,Mary admires John
Mary is in love with John,26.029047,-115.752205,Mary is in love with John
Mary respects John,68.931564,226.243515,Mary respects John
John admires Mary,-140.291473,186.148941,John admires Mary
John is in love with Mary,-166.802856,-25.228626,John is in love with Mary


In [28]:
interativePlot(df_sentences_emb)

## Exemplo 2

In [29]:

examples = ['I was given a card by her in the garden',
 'In the garden , she gave me a card',
 'She gave me a card in the garden',
 'She was given a card by me in the garden',
 'In the garden , I gave her a card',
 'I gave her a card in the garden']

In [30]:
h,c = seq2seq.get_sentences_embeddings(model, examples, 'en', device)

/usr/local/lib/python3.8/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [31]:
sentences_emb = ((h+c)/2).mean(0)
sentences_emb = sentences_emb.cpu().detach().numpy()
sentences_emb.shape

(6, 512)

In [32]:
from sklearn.manifold import TSNE
tsne = TSNE(metric = 'cosine')
reduced_sentences_emb = tsne.fit_transform(sentences_emb)
reduced_sentences_emb.shape

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


(6, 2)

In [33]:
sentences_emb = {}
for idx, sentence in enumerate(examples):
  sentences_emb[sentence] = reduced_sentences_emb[idx]
df_sentences_emb = pd.DataFrame.from_dict(data=sentences_emb, columns=['x', 'y'], orient='index')

In [34]:
df_sentences_emb['word'] = df_sentences_emb.index
df_sentences_emb.head()

,x,y,word
I was given a card by her in the garden,-193.406158,-98.960007,I was given a card by her in the garden
"In the garden , she gave me a card",196.171844,129.093994,"In the garden , she gave me a card"
She gave me a card in the garden,21.613659,-30.440687,She gave me a card in the garden
She was given a card by me in the garden,197.516846,-118.057922,She was given a card by me in the garden
"In the garden , I gave her a card",-111.999184,147.692154,"In the garden , I gave her a card"


In [35]:
interativePlot(df_sentences_emb)